In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
# from selenium.webdriver import EdgeOptions
from selenium.webdriver import ChromeOptions
import time 

In [2]:
# 下載圖片用的程式
import os
import requests
# download and save a given image
def __download_and_save_image(link, directory, src='fotolia'):
	print("Attempting to download: " + link)
	r = requests.get(link)
	if r.status_code == 200:
		
		# depends on source
		if src == 'fotolia':
			try:
				filename = r.headers['Content-Disposition'].split('filename="')[1][:-2]
			except:
				print("No Content-Disposition header present.")
				return
		elif src == 'istock':
			try:
				filename = r.headers['Content-Disposition'].split('filename=')[1]
			except:
				print("No Content-Disposition header present.")
				return
		if not os.path.isdir(directory):
			os.mkdir(directory)
		filename = os.sep.join([directory, filename])
		print("Saving to filename: %s "%(filename))
		with open(filename, 'wb') as f:
			f.write(r.content)
	else:
		print("Couldn't download from link: " + link)

In [3]:
# 網頁自動化用的程式
option = ChromeOptions()
option.add_experimental_option('excludeSwitches', ['enable-automation'])
prefs = {"profile.default_content_setting_values.notifications" : 2}
option.add_experimental_option("prefs",prefs)

In [4]:
from tqdm import tqdm
# 抓下來的圖片存放的位置
dir = "mountain_real"
# 想要抓取的圖片類型
image_type = "mountain real"
# 想要抓取的圖片數量
num_of_images = 100

# 沒有浮水印的圖片
url = f"https://www.istockphoto.com/search/2/image-film?phrase={image_type}"
# 這邊會自動開啟瀏覽器，程式會自己跑，不要把瀏覽器關掉
driver = webdriver.Chrome(options=option)
driver.get(url)

time.sleep(2)
images = driver.find_elements(By.XPATH, '//img[@class="yGh0CfFS4AMLWjEE9W7v"]')
downloaded = 0

first_page_images = len(images)
i=0
page = 1
print(f"crawling page {page}")
while(images):
    image = images.pop()
    src = image.get_attribute('src')
    time.sleep(0.5)
    __download_and_save_image(src, dir, 'istock')
    downloaded += 1
    i+=1
    if i == first_page_images - 1:
      i=0
      driver.execute_script("document.querySelector('[title=\"Next page\"]').click()")  
      page+=1
      print(f"crawling page {page}")
      time.sleep(2)
      images = driver.find_elements(By.XPATH, '//img[@class="yGh0CfFS4AMLWjEE9W7v"]')
    if downloaded >= num_of_images:
      break
    
    # print(i, src)

crawling page 1
Attempting to download: https://media.istockphoto.com/id/1441906287/photo/man-hiking-on-the-background-of-interlaken-in-swiss-alps.jpg?s=612x612&w=0&k=20&c=JZC51LFohYUENCatf9-xuUUZ_F-0agbL9zo4Ha2BgI0=
Saving to filename: mountain real\istockphoto-1441906287-612x612.jpg 
Attempting to download: https://media.istockphoto.com/id/1282106398/photo/cycling-on-north-uist-scotland.jpg?s=612x612&w=0&k=20&c=vNXCT70h7XRWpym7ymvf2BL_OlDYg4G7araZ6DmiL7k=
Saving to filename: mountain real\istockphoto-1282106398-612x612.jpg 
Attempting to download: https://media.istockphoto.com/id/1399438901/photo/hiking-couple-resting-after-walking-and-drinking-hot-beverage.jpg?s=612x612&w=0&k=20&c=-Yl1XULgZZxqDqIUJ-ufEfGw797yAtadm_VqjUttwV4=
Saving to filename: mountain real\istockphoto-1399438901-612x612.jpg 
Attempting to download: https://media.istockphoto.com/id/1098245018/photo/happy-woman-hiking.jpg?s=612x612&w=0&k=20&c=zzcsA0qvBAkr5sfwg6KTBGt2hLGwlcJoQLkq7nxKq-s=
Saving to filename: mountain 

In [6]:
from PIL import Image
import random

def add_watermark(base_image, output_image_path, watermark, opacity, random_position = False, watermark_pos = (0, 0)):
    # 開啟圖片
    # base_image = Image.open(input_image_path).convert("RGB")
    # watermark = Image.open(watermark_image_path).convert("RGBA")
    x = watermark_pos[0]
    y = watermark_pos[1]
    # watermark.resize()
    # 計算浮水印的大小
    watermark_width, watermark_height = watermark.size
    # 確認座標是否在圖片範圍內
    base_width, base_height = base_image.size
    if random_position:
        x = random.randint(0, base_width - watermark_width)
        y = random.randint(0, base_height - watermark_height)
    if x < 0 or x > base_width - watermark_width or y < 0 or y > base_height - watermark_height:
        raise ValueError("Coordinates are outside the image boundaries.")
    # 複製原始圖片
    watermarked_image = base_image.copy()
    # 將浮水印貼到圖片上
    watermarked_image.paste(watermark, (x, y), mask=watermark)
    # 調整透明度
    watermarked_image = Image.blend(base_image, watermarked_image, opacity)
    # 儲存新圖片
    watermarked_image.save(output_image_path)
    watermarked_image.close()
    base_image.close()

In [7]:
def resize_with_black_fill(image_path, target_width, target_height, output=False, output_path=None):
    # 打開圖片
    original_image = Image.open(image_path)
    
    # 原始圖片的寬度和高度
    original_width, original_height = original_image.size
    
    # 計算縮放後的大小
    target_ratio = target_width / target_height
    
    # 原始圖片的寬高比例
    original_ratio = original_width / original_height
    
    # 計算縮放比例
    if original_ratio > target_ratio:
        # 如果原始圖片的寬高比例大於目標寬高比例，則以目標寬度為準進行縮放
        new_width = target_width
        new_height = int(target_width / original_ratio)
    else:
        # 否則以目標高度為準進行縮放
        new_height = target_height
        new_width = int(target_height * original_ratio)
    
    # 縮放圖片
    resized_image = original_image.resize((new_width, new_height))
    
    # 在黑色背景上創建一個新的畫布
    final_image = Image.new("RGB", (target_width, target_height), "black")
    
    # 將縮放後的圖片粘貼到新的畫布上，使其置中
    paste_x = (target_width - new_width) // 2
    paste_y = (target_height - new_height) // 2
    final_image.paste(resized_image, (paste_x, paste_y))
    
    # 儲存結果
    # check if the output directory exists, if not create it
    if output:
        if not os.path.isdir(os.path.dirname(output_path)):
            os.makedirs(os.path.dirname(output_path))
        final_image.save(output_path)
    return final_image

In [4]:
resize_with_black_fill("mountain_real/istockphoto-516427571-612x612.jpg", 480, 360)

NameError: name 'Image' is not defined

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
def crop_watermark(watermark_path, output_path, resize_ratio=0.2):
    watermark = Image.open(watermark_path).convert("RGBA")
    watermark = watermark.resize((int(watermark.width * resize_ratio), int(watermark.height * resize_ratio)))
    watermark_arr = np.array(watermark)
    x,y = np.where(watermark_arr[:,:,3] != 0)
    xmin, xmax, ymin, ymax = np.min(x), np.max(x), np.min(y), np.max(y)
    watermark_arr = watermark_arr[xmin:xmax, ymin:ymax]
    Image.fromarray(watermark_arr).convert("RGBA").save(output_path)
    
crop_watermark("./watermark_img/hero-image.png", "./watermark_img/watermark.png")

NameError: name 'Image' is not defined

In [62]:
Image.open("./watermark_img/watermark.png").size

(159, 61)

In [8]:
# 添加浮水印
import os
from PIL import Image

input_image_folder = "./mountain_real"
output_image_folder = "./mountain_real/watermarked_alpha2"
watermark_img_path = "./watermark_img/watermark.png"
if not os.path.exists(output_image_folder):
    os.makedirs(output_image_folder)

for img in os.listdir(input_image_folder):
    print(f"add watermark to {img}")
    input_image_path = os.path.join(input_image_folder, img)
    output_image_path = os.path.join(output_image_folder, img)
    watermark = Image.open(watermark_img_path).convert("RGBA")
    if os.path.isdir(input_image_path):
        continue
    # add_watermark(input_image_path, output_image_path, watermark_img_path, 0.5, (160, 120), True)
    add_watermark(resize_with_black_fill(input_image_path, 480, 360), output_image_path, watermark, 0.2, True, (100, 100))

add watermark to istockphoto-1041742312-612x612.jpg
add watermark to istockphoto-1041972386-612x612.jpg
add watermark to istockphoto-1058409750-612x612.jpg
add watermark to istockphoto-1060529042-612x612.jpg
add watermark to istockphoto-1098245018-612x612.jpg
add watermark to istockphoto-1170633968-612x612.jpg
add watermark to istockphoto-1184901438-612x612.jpg
add watermark to istockphoto-1191175163-612x612.jpg
add watermark to istockphoto-1197801309-612x612.jpg
add watermark to istockphoto-1215540142-612x612.jpg
add watermark to istockphoto-1217879619-612x612.jpg
add watermark to istockphoto-1244723650-612x612.jpg
add watermark to istockphoto-1259152913-612x612.jpg
add watermark to istockphoto-1281066419-612x612.jpg
add watermark to istockphoto-1281528222-612x612.jpg
add watermark to istockphoto-1282106398-612x612.jpg
add watermark to istockphoto-1285662078-612x612.jpg
add watermark to istockphoto-1287936962-612x612.jpg
add watermark to istockphoto-1288811952-612x612.jpg
add watermar